In [1]:
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException,NoSuchFrameException, WebDriverException, UnexpectedAlertPresentException
import time
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [3]:
ca = pd.read_excel('1_이미용_카테고리2.xlsx')
ca.head()

,대카테고리,카테고리
0,기초 화장품,기초 화장품 세트
1,기초 화장품,마스크팩
2,기초 화장품,비누
3,기초 화장품,선크림
4,기초 화장품,세럼


In [2]:
# path 설정
driver = Chrome('C:/Users/User/공공 빅데이터 인턴십_공영홈쇼핑/chromedriver.exe')

## 1. 블로그 포스트 url

In [5]:
url_list = []
def get_url_list(category_input):    
    # page
    for page_num_input in tqdm(range(1,1001, 10)):
        url = 'https://search.naver.com/search.naver?date_from=20190101&date_option=8&date_to=20200930&dup_remove=1&nso=p%3Afrom20190101to20200930&post_blogurl=&post_blogurl_without=&query={category}&sm=tab_pge&srchby=all&st=sim&where=post&start={page_num}'.format(category = category_input, page_num = page_num_input)
        driver.get(url)
        time.sleep(1)
        
        # 블로그 포스트 url 저장
        blog_post_url = driver.find_elements_by_class_name('sh_blog_title._sp_each_url._sp_each_title')
        for blog_post in blog_post_url:
            url_list.append([category_input, blog_post.text, blog_post.get_attribute('href')])

In [6]:
from tqdm.notebook import tqdm

for category in tqdm(ca['카테고리']):
    get_url_list(category)

In [7]:
blog_info = pd.DataFrame(url_list, columns = ['카테고리','제목', '링크'])
len(blog_info)

17000

In [8]:
blog_info.to_excel('1_트렌드분석_블로그url2.xlsx', index = False)

## 2. url접속 -> 내용 크롤링

In [4]:
blog_info = pd.read_excel('1_트렌드분석_블로그url2.xlsx')
blog_info.head()

,카테고리,제목,링크
0,기초 화장품 세트,미스랩 갈락토미세스 기초화장품세트 할머니도 만족하셔요!,https://blog.naver.com/ik0726?Redirect=Log&log...
1,기초 화장품 세트,기초화장품 세트 순하고 생기있어보여요,https://blog.naver.com/blood1226?Redirect=Log&...
2,기초 화장품 세트,남자기초화장품 세트제품으로 한방에 관리,https://blog.naver.com/lsi9094?Redirect=Log&lo...
3,기초 화장품 세트,기초화장품 내 피부에 딱 맞게 물들임 세트로 골랐어요,https://blog.naver.com/greenjbee?Redirect=Log&...
4,기초 화장품 세트,기초화장품세트 건조함도 끄떡없지,https://kanalia250.blog.me/222098278149


In [5]:
blog_info2 = blog_info.copy()
blog_info2['내용'] = np.nan
blog_info2['날짜'] = np.nan
blog_info2.head()

,카테고리,제목,링크,내용,날짜
0,기초 화장품 세트,미스랩 갈락토미세스 기초화장품세트 할머니도 만족하셔요!,https://blog.naver.com/ik0726?Redirect=Log&log...,NaN,NaN
1,기초 화장품 세트,기초화장품 세트 순하고 생기있어보여요,https://blog.naver.com/blood1226?Redirect=Log&...,NaN,NaN
2,기초 화장품 세트,남자기초화장품 세트제품으로 한방에 관리,https://blog.naver.com/lsi9094?Redirect=Log&lo...,NaN,NaN
3,기초 화장품 세트,기초화장품 내 피부에 딱 맞게 물들임 세트로 골랐어요,https://blog.naver.com/greenjbee?Redirect=Log&...,NaN,NaN
4,기초 화장품 세트,기초화장품세트 건조함도 끄떡없지,https://kanalia250.blog.me/222098278149,NaN,NaN


In [261]:
# blog_info2 = blog_info2[blog_info2['카테고리']!='빗'].reset_index(drop = True)
# len(blog_info2)

16000

In [5]:
li = range(0,16000,1000) #1000개마다 저장하기 위한 인덱스 리스트

for num in tqdm(li):
    print(blog_info2['카테고리'][num])
    df_1000 = blog_info2.iloc[num:num+1000,:].reset_index()
    
    for i in tqdm(range(len(df_1000))):
        link = df_1000['링크'][i]   
        
        try: 
            driver.get(link)    
            time.sleep(1)
            try:
                driver.switch_to.frame('mainFrame')

                ################ case1 #################
                try:
                    try:
                        text = driver.find_element_by_class_name('se-component.se-text.se-l-default')        
                        texts = driver.find_elements_by_class_name('se-component.se-text.se-l-default')
                        content_list = []

                        for text in texts:
                            content_list.append(text.text)
                        content_str = ' '.join(content_list)

                        df_1000['내용'][i] = content_str
                        time.sleep(0.5)

                        date_text = driver.find_element_by_class_name('se_publishDate.pcol2').text
                        df_1000['날짜'][i] = date_text
                    except NoSuchElementException:
                        pass

                ################ case2 #################
                except:
                    try:
                        texts = driver.find_element_by_id('postViewArea').text

                        df_1000['내용'][i] = texts
                        time.sleep(0.5)

                        date_text = driver.find_element_by_class_name('date.fil5.pcol2._postAddDate').text
                        df_1000['날짜'][i] = date_text
                    except NoSuchElementException:
                        pass
            except UnexpectedAlertPresentException:
                pass            
                  
        except TimeoutException:
            pass
        except NoSuchFrameException:
            pass
        except WebDriverException:
            pass

    df_1000.to_excel('1_트렌드분석_블로그내용_{name}.xlsx'.format(name = df_1000['카테고리'][i]))

기초 화장품 세트


KeyboardInterrupt: 

### -----------------------------------------------------------------------------------------------------------------------------------------------------------